In [ ]:
%pip install openai
%pip install azure-ai-formrecognizer
%pip install langchain
%pip install pandas

Import needed packages 

In [ ]:
import openai
import requests
import json
import io
import os
import pandas as pd
import math
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import DocumentAnalysisClient
import os
from openai import AzureOpenAI

In [ ]:
# extracts text from pdf
def analyze_pdf_with_form_recognizer(endpoint, api_key, file_path):
    credential = AzureKeyCredential(api_key)
    document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(api_key)
    )

    with open(file_path, "rb") as file:
        poller = document_analysis_client.begin_analyze_document("prebuilt-layout", file) #"prebuild-layout"is predefined Azure DocumentAnalysisClient ML Model 
        result = poller.result()

    return result

# Generate complete text from AnalyzeResult
def complete_text_from_analyze_result(result):
    complete_text = ""
    for paragraph in result.paragraphs:
        if paragraph.role != 'pageFooter' and paragraph.role != 'pageHeader':
            recognized_text = paragraph.content
            complete_text += recognized_text
    return complete_text

In [ ]:
# Azure Forms Recognition API endpoint and key
endpoint = "REPLACE_WITH_YOUR_OWN" ##### here should be the endpoint of your azure subscription's OpenAI instance. 
api_key = "REPLACE_WITH_YOUR_OWN" ##### here you will need to modify to your OpenAI apikey, 

# Azure OpenAI info
client = AzureOpenAI(
  azure_endpoint = "REPLACE_WITH_YOUR_OWN",##### here should be the endpoint of your azure subscription's OpenAI instance. 
  api_key="REPLACE_WITH_YOUR_OWN", ##### here you will need to modify to your OpenAI aipkey, 
  api_version="2023-05-15" ##### you can choose another version, you might need to find the version name in the documentation. 
)

# pdf_file_path = "15.pdf" ###### where you local pdf file you want to study, 
# result = analyze_pdf_with_form_recognizer(endpoint, api_key, pdf_file_path)
# print(result)

In [ ]:
def get_pdf_files(folder_path):
    pdf_files = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(folder_path, file))
    return pdf_files


In [ ]:
research_description = "Analysis of materials for building efficient solar cells, and building an understanding of molecular structure on PCE values."

main_prompt = f"""
        You are a polymer scientist analyzing scientific papers. You want will give a relevance score between 0 and 1 to the provided scientific paper, with 0 meaning completely irrelevant and 1 being extremely relevant. 
         Relevance is based on whether the paper matches the following research topic: {research_description}.
         In addition, you will provide a list of polymer names discussed in the paper. 
         You must give the response as a valid json structure, with 2 elements: score (which is going to be a double value between 0 and 1), and polymers (which is going to be an array of string containing the names of the polymers discussed in the paper).
        """

In [ ]:
papers_to_analyze = get_pdf_files("./PapersToAnalyze")

In [ ]:
papers_to_analyze

In [ ]:
results = []

for paper in papers_to_analyze:

    # First read the pdf 
    fr_result = analyze_pdf_with_form_recognizer(endpoint, api_key, paper)
    fulltext = complete_text_from_analyze_result(fr_result)

    response = client.chat.completions.create(
        model="REPLACE_WITH_YOUR_OWN", # Name of your deployment on https://oai.azure.com/ 
        messages=[
            {"role": "system", "content": main_prompt}, 
            {"role": "user", "content": f"Here is the paper to analyze: {fulltext}"},
        ],
        temperature = 0
        )
    
    results.append({'fileName': os.path.basename(paper), 'score': response.choices[0].message})
    print(os.path.basename(paper))
    print(f"Score: {response.choices[0].message.content}")
    print()
    ###results_df = results_df.append({'fileName': os.path.basename(paper), 'score': response.choices[0].message}, ignore_index=True)
    
    
